In [42]:
%reload_ext autoreload
%autoreload 2

In [43]:
import os
import pandas as pd
from different_lineage import different_lineage

In [ ]:
data_dir = "data_4_28_25/"
files =  sorted(list(os.listdir(data_dir)))
before_noise_files = [file for file in files if file.endswith("before_noise")]
gt_files = [file for file in files if file.endswith("ground_truth_matrix")]
print(before_noise_files[:2])
print(gt_files[:2])

['simNo_1-n_100-m_100-fp_0-fn_0.05-na_0.SC.before_noise', 'simNo_1-n_100-m_100-fp_0-fn_0.1-na_0.SC.before_noise']
['simNo_1-n_100-m_100.ground_truth_matrix', 'simNo_1-n_100-m_1000.ground_truth_matrix']


In [ ]:
# Double check that .before_noise and .ground_truth_matrix files are the same
for i in range(1, 6):
    before_noise_ex = os.path.join(data_dir, f"simNo_{i}-n_100-m_100-fp_0-fn_0.05-na_0.SC.before_noise")
    gt_ex = os.path.join(data_dir, f"simNo_{i}-n_100-m_100.ground_truth_matrix")
    comparison_results = []
    with open(before_noise_ex, 'rt') as bn_file:
        with open(gt_ex, 'rt') as gt_ex_file:
            comparison_results.append(bn_file.read() == gt_ex_file.read())
    assert all(comparison_results), f"{comparison_results=} failed for some simulation(s)"

In [ ]:
def get_matrix(matrix_path):
    df_input = pd.read_csv(matrix_path, delimiter="\t", index_col=0)
    return df_input.values
before_noise_ex = os.path.join(data_dir, f"simNo_{i}-n_100-m_100-fp_0-fn_0.05-na_0.SC.before_noise")
out_matrix = get_matrix(before_noise_ex)

<class 'numpy.ndarray'>


In [21]:
fp_csv = pd.read_csv("false_positive_12_2_25/false_positive/false_positive_results.csv")

In [71]:
data_dir = "data_4_28_25/"
solutions_dir = os.path.join("false_positive_12_2_25", "false_positive", "solutions")
diff_lin_numerators, diff_lin_denominators = [], []
# num_tries = 10
for idx, row in fp_csv.iterrows():
    # if not num_tries:
    #     break

    sim_name = row["sim_name"]
    # sim_name = "simNo_5-n_50-m_20-fp_0-fn_0.05-na_0.SC" # TODO: Delete me
    solution_path = os.path.join(solutions_dir, sim_name + ".CFMatrix")
    solution_path = solution_path.replace("fp_0-", "fp_0.0-")
    gt_path = os.path.join(data_dir, sim_name + ".before_noise")



    sol_matrix = get_matrix(solution_path)
    gt_matrix = get_matrix(gt_path)
    diff_lin_numerator, diff_lin_denominator = different_lineage(sol_matrix, gt_matrix)
    diff_lin_numerators.append(diff_lin_numerator)
    diff_lin_denominators.append(diff_lin_denominator)
    diff_lin_score = diff_lin_numerator / diff_lin_denominator
    # if diff_lin_score < 1:
    #     print(diff_lin_score)
    #     num_tries -= 1

In [68]:
import numpy as np
diff_lin_scores = np.array(diff_lin_numerators) / np.array(diff_lin_denominators)

In [70]:
diff_lin_scores[diff_lin_scores != 1]

array([0.89634146, 0.89634146, 0.89634146, 0.87662338, 0.87662338,
       0.87662338, 0.89261745, 0.89261745, 0.89261745, 0.87234043])

In [72]:
fp_csv["different_lineage_numerator"] = diff_lin_numerators
fp_csv["different_lineage_denominator"] = diff_lin_denominators
fp_csv.to_csv("false_positive_12_2_25/false_positive/false_positive_results_with_different_lineage.csv")